# NHL - Predicting Wins

### Overview

One of my previous portfolio pieces include an analysis of the NHL since 1964 to present. When I first undertook the piece I conducted the analysis using R, and it was done in my early days in data science. It has used both supervised and unsupervised machine learning methods for the predictive analytic portion.

I am now redoing this piece, and this is just the first part of a multi faceted project that I am working on. It is my most favorite data to work with because of my love for hockey. Analytics have proven successful in other sports like baseball, however I have always believed that due to the "bounce of the puck on a slippery surface" it is VERY difficult to predict an outcome. That is my personal take on it, however, I want to prove this wrong and show that we are able to provide directionality thoroughout each part of this project. 

For this project I zeroed in from 1997 to present. This portion looks at predicting wins, so that I may be able to  help the number to give insight on the number of wins a team may finish with in a season. 

#### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

C:\Users\willjdsouza\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


#### Uploading data and visualizing it

In [2]:
np.random.RandomState(40)
teams = pd.read_csv("C:/Users/willjdsouza/windows_stuff/Desktop/hockey_final.csv", encoding='latin-1')

In [3]:
teams.head()

,Team,Divison,Season,GP,W,L,T,OT,P,ROW,...,GF/GP,GA/GP,PP%,PK%,S/GP,SA/GP,FOW%,W/GP,L/GP,P/GP
0,Anaheim Ducks,PAC,1997,82,26,43,13,NaN,65,0,...,2.50,3.18,0.12,0.82,26.6,30.4,51.3,0.32,0.52,0.79
1,Arizona Coyotes,PAC,1997,82,35,35,12,NaN,82,0,...,2.73,2.77,0.15,0.84,27.7,27.1,50.0,0.43,0.43,1.00
2,Boston Bruins,ATL,1997,82,39,30,13,NaN,91,0,...,2.70,2.37,0.17,0.85,27.2,26.4,51.9,0.48,0.37,1.11
3,Buffalo Sabres,ATL,1997,82,36,29,17,NaN,89,0,...,2.57,2.28,0.13,0.84,26.5,31.2,47.3,0.44,0.35,1.09
4,Calgary Flames,PAC,1997,82,26,41,15,NaN,67,0,...,2.65,3.07,0.12,0.84,27.7,27.8,46.9,0.32,0.50,0.82


In [4]:
teams.shape

(592, 24)

In [5]:
teams.describe()

,Season,GP,W,L,T,OT,P,ROW,P%,GF,...,GF/GP,GA/GP,PP%,PK%,S/GP,SA/GP,FOW%,W/GP,L/GP,P/GP
count,592.000000,592.000000,592.000000,592.000000,592.000000,539.000000,592.000000,592.000000,592.000000,592.000000,...,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000,592.000000
mean,2007.309122,80.277027,38.278716,31.005068,3.719595,7.988868,87.550676,23.021959,0.546250,219.231419,...,2.729307,2.729291,0.176014,0.825186,29.288682,29.287500,49.990878,0.477230,0.386571,1.091199
std,6.115420,7.463682,8.631682,7.850225,5.493182,3.561445,16.786732,17.827179,0.092118,32.931640,...,0.311560,0.343507,0.029611,0.029736,2.195702,2.482626,2.073061,0.099502,0.091092,0.184395
min,1997.000000,48.000000,14.000000,7.000000,0.000000,0.000000,36.000000,0.000000,0.240000,109.000000,...,1.830000,1.890000,0.090000,0.730000,23.700000,22.100000,44.100000,0.170000,0.150000,0.480000
25%,2002.000000,82.000000,32.000000,26.000000,0.000000,5.000000,77.000000,0.000000,0.480000,204.750000,...,2.520000,2.480000,0.160000,0.810000,27.800000,27.700000,48.600000,0.410000,0.330000,0.960000
50%,2008.000000,82.000000,39.000000,30.000000,0.000000,8.000000,90.500000,29.000000,0.560000,221.000000,...,2.715000,2.710000,0.175000,0.830000,29.200000,29.400000,50.000000,0.490000,0.375000,1.120000
75%,2013.000000,82.000000,45.000000,36.000000,9.000000,11.000000,100.000000,38.000000,0.610000,241.000000,...,2.940000,2.950000,0.190000,0.850000,30.700000,30.900000,51.300000,0.550000,0.440000,1.220000
max,2017.000000,82.000000,58.000000,57.000000,20.000000,18.000000,124.000000,54.000000,0.800000,313.000000,...,3.820000,3.820000,0.270000,0.900000,36.200000,35.900000,56.400000,0.750000,0.700000,1.600000


They are are a number of ways I looked at the data, however I wanted to share a few visualizations I created with Kibana. I am becoming a huge fan of the elasticstack and am looking to work with it more. The visualizations are meant to be interactive, but I had to share static screenshots because it is quite difficult to share the interactive visualizations

The first 4 charts I looked at the divisions as a whole and measured different metrics. The next 4 I looked at certain metrics given the teams in a division

<img src="Division.W_GP.PNG">
<img src="Division.L_GP.PNG">
<img src="Division.OTW.PNG">
<img src="Division.P_GP.PNG">

<img src="Team.MET.PNG">
<img src="Team.CEN.PNG">
<img src="Team.PAC.PNG">
<img src="Team.ATL.PNG">

The major point to share after looking at these visualizations can be summed up in one major point. The data is not only well distributed, it is also almost identical when comparing different groups. We dont see many spikes or outliers in the data which means that our predictions may come out quite well. 

When we segment the data, the magnitude of variances amongst the groups do not deviate from eachother greatly, so we can also that the leauge is highly competitive. Teams average out to having similar statistics, which points out how effective the salary cap system and talent is distributed amongst the teams in the leauge.

#### Correlations & training data

In [6]:
teams.corr(method='pearson')

,Season,GP,W,L,T,OT,P,ROW,P%,GF,...,GF/GP,GA/GP,PP%,PK%,S/GP,SA/GP,FOW%,W/GP,L/GP,P/GP
Season,1.000000,-0.177373,0.181012,-0.136116,-0.809865,0.505115,0.077867,0.789105,0.181496,-0.055463,...,0.065954,0.060104,0.386091,-0.401304,0.465158,0.411701,-0.001926,0.272418,-0.075458,0.184113
GP,-0.177373,1.000000,0.382519,0.365407,0.156578,0.176640,0.470428,0.029254,-0.026148,0.644819,...,0.055398,0.050234,-0.051956,0.053287,0.016009,0.013428,0.000098,-0.052917,-0.001935,-0.027757
W,0.181012,0.382519,1.000000,-0.616054,-0.233553,0.017296,0.972366,0.501531,0.882501,0.752254,...,0.679852,-0.557382,0.481532,0.271103,0.474018,-0.266050,0.244229,0.897587,-0.807669,0.882853
L,-0.136116,0.365407,-0.616054,1.000000,0.061754,0.064225,-0.621951,-0.243360,-0.904671,-0.235125,...,-0.591029,0.699828,-0.451655,-0.323976,-0.391293,0.398958,-0.276169,-0.832065,0.922180,-0.904485
T,-0.809865,0.156578,-0.233553,0.061754,1.000000,-0.577276,-0.079354,-0.875922,-0.170142,-0.034324,...,-0.165546,-0.183196,-0.349126,0.354484,-0.442235,-0.414984,0.001751,-0.317896,0.005886,-0.174650
OT,0.505115,0.176640,0.017296,0.064225,-0.577276,1.000000,0.061431,0.542787,-0.039366,0.085937,...,-0.037446,0.155598,0.127578,-0.258787,0.305755,0.334228,0.016708,-0.074660,-0.005045,-0.037842
P,0.077867,0.470428,0.972366,-0.621951,-0.079354,0.061431,1.000000,0.384699,0.865218,0.783459,...,0.650600,-0.591539,0.435716,0.314598,0.433842,-0.317402,0.254774,0.827119,-0.847414,0.864596
ROW,0.789105,0.029254,0.501531,-0.243360,-0.875922,0.542787,0.384699,1.000000,0.412881,0.283759,...,0.346541,-0.012981,0.454752,-0.248225,0.550003,0.312099,0.056723,0.522691,-0.275229,0.416673
P%,0.181496,-0.026148,0.882501,-0.904671,-0.170142,-0.039366,0.865218,0.412881,1.000000,0.521767,...,0.703547,-0.704378,0.519115,0.333571,0.479646,-0.371044,0.289894,0.970370,-0.963586,0.999291
GF,-0.055463,0.644819,0.752254,-0.235125,-0.034324,0.085937,0.783459,0.283759,0.521767,1.000000,...,0.796320,-0.073855,0.403668,0.063637,0.397408,-0.083775,0.118000,0.506965,-0.502532,0.520939


In [15]:
X = teams[['GF/GP', 'GA/GP', 'PP%', 'PK%', 'S/GP']] #features for training
y = teams.loc[:, ['W']] #target variable

X_train_prepared, X_test_prepared, y_train, y_test,  = train_test_split(X, y, test_size=0.15, random_state=42) #random split for train and test sets

print("X_train length:", len(X_train_prepared))
print("X_test length:", len(X_test_prepared))
print("y_train length:", len(y_train))
print("y_test length:", len(y_test))

X_train length: 503
X_test length: 89
y_train length: 503
y_test length: 89


In [13]:
X_train_prepared.dtypes

GF         int64
GF/GP    float64
GA/GP    float64
PP%      float64
PK%      float64
S/GP     float64
dtype: object

In [8]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
    

In [9]:
#Building a full pipeline for data transformation

#numerical_values = list(teams[['GF', 'GF/GP', 'GA/GP', 'PP%', 'PK%', 'S/GP']])

#catergorical_values = teams[['Team', 'Divison']]

#num_pipeline = Pipeline([
  #      ('selector', DataFrameSelector(numerical_values)),
 #       ('std_scaler', StandardScaler()), #scaling data using a standard scaler
#    ])

#cat_pipeline = Pipeline([
#        ('selector', DataFrameSelector(catergorical_values)),
#        ('cat_encoder', OneHotEncoder(sparse=)),
#    ]) 


#full_pipeline = FeatureUnion(transformer_list=[
#        ("num_pipeline", num_pipeline),
#        ("cat_pipeline", cat_pipeline),
#])

In [10]:
#from sklearn.pipeline import FeatureUnion

#X_train_prepared = full_pipeline.fit_transform(X_train) 
#X_test_prepared = full_pipeline.transform(X_test)

In [11]:
#X_train_prepared

##### Shortlisting models 

In [12]:
lin_r = LinearRegression()
lin_r.fit(X_train_prepared, y_train)
lin_r_predict = lin_r.predict(X_train_prepared)
lin_r_scores = mean_squared_error(lin_r_predict, np.ravel(y_train))



las = Lasso()
las.fit(X_train_prepared, y_train)
las_predict = las.predict(X_train_prepared)
las_scores = mean_squared_error(las_predict, np.ravel(y_train))


sgd = SGDRegressor()
sgd.fit(X_train_prepared, y_train)
sgd_predict = sgd.predict(X_train_prepared)
sgd_scores = mean_squared_error(sgd_predict, np.ravel(y_train))


dtr = DecisionTreeRegressor()
dtr.fit(X_train_prepared, y_train)
dtr_predict = dtr.predict(X_train_prepared)
dtr_scores = mean_squared_error(dtr_predict, np.ravel(y_train))


rfr = RandomForestRegressor()
rfr.fit(X_train_prepared, y_train)
rfr_predict = rfr.predict(X_train_prepared)
rfr_scores = mean_squared_error(rfr_predict, np.ravel(y_train))


ada_rfr = AdaBoostRegressor(rfr)
ada_rfr.fit(X_train_prepared, y_train)
ada_rfr_predict = ada_rfr.predict(X_train_prepared)
ada_rfr_scores = mean_squared_error(ada_rfr_predict, np.ravel(y_train))


ada_dtr = AdaBoostRegressor(dtr)
ada_dtr.fit(X_train_prepared, y_train)
ada_dtr_predict = ada_dtr.predict(X_train_prepared)
ada_dtr_scores = mean_squared_error(ada_dtr_predict, np.ravel(y_train))

print("Linear Regression Scores - R2 Score:", r2_score(lin_r_predict, np.ravel(y_train)), "RMSE:", 
      np.sqrt(lin_r_scores))

print("Lasso Scores - RMSE:", np.sqrt(las_scores))
      
print("SGD Scores - RMSE:", np.sqrt(lin_r_scores))

print("Decision Tree Scores - RMSE:", np.sqrt(dtr_scores))

print("Random Forest Scores - RMSE:", np.sqrt(rfr_scores))

print("Adaboost (Random Forest) Scores - RMSE:", np.sqrt(ada_rfr_scores))

print("Adaboost (Decision Trees) Scores - RMSE:", np.sqrt(ada_dtr_scores))

C:\Users\willjdsouza\Anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\willjdsouza\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\willjdsouza\Anaconda\lib\site-packages\ipykernel_launcher.py:27: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\will

Linear Regression Scores - R2 Score: 0.8378634286978851 RMSE: 3.2372092753413644
Lasso Scores - RMSE: 4.473793839362244
SGD Scores - RMSE: 3.2372092753413644
Decision Tree Scores - RMSE: 0.0
Random Forest Scores - RMSE: 1.6037295003020817
Adaboost (Random Forest) Scores - RMSE: 1.015701776804382
Adaboost (Decision Trees) Scores - RMSE: 0.16076358548345154


C:\Users\willjdsouza\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


During the shortlisting phase, it looks like all models do fairly well. The decision trees fit the data tremendously well, however, I can easily tell that this will cause overfitting. 

#### Hypertuning random forest

In [13]:

from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [8, 10, 12,],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf':[1, 3, 5]}
  ]

rfrc = RandomForestRegressor()
grid_search = GridSearchCV(rfr, param_grid,
scoring='neg_mean_squared_error')
grid_search.fit(X_train_prepared, np.ravel(y_train))
print(grid_search.best_params_)
print(np.sqrt(-grid_search.best_score_))


{'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 12}
3.6572469973649473


After many attempts at hypertuning, it was difficult to find any sort of gain with it. Turns out the default paramaeters itself work out nicely.

#### Test set and comaprisons

Next, I compared the variancse between 4 regressors (random forest, decision tree, linear regression and adaboost). I wanted to see which regressor generalized well so I could choose the best one

In [14]:
rfr_predict = rfr.predict(X_test_prepared)
rfr_test_scores = mean_squared_error(rfr_predict, np.ravel(y_test))

dtr_predict= dtr.predict(X_test_prepared)
dtr_test_scores = mean_squared_error(dtr_predict, np.ravel(y_test))
    
lin_r_predict = lin_r.predict(X_test_prepared)
lin_r_test_scores = mean_squared_error(lin_r_predict, y_test)
    
ada_dtr_predict = ada_dtr.predict(X_test_prepared)
ada_dtr_test_scores = mean_squared_error(ada_dtr_predict, np.ravel(y_test))
    
ada_rfr_predict = ada_rfr.predict(X_test_prepared)
ada_rfr_test_scores = mean_squared_error(ada_rfr_predict, np.ravel(y_test))
    
d = pd.DataFrame({'Method': ['Linear Regression', 'Decision Trees', 'Adaboost (Decision Trees)',
                             'Random Forest', 'Adaboost (Random Forest)'], 
                  'Training Scores': [np.sqrt(lin_r_scores), np.sqrt(dtr_scores), np.sqrt(ada_dtr_scores), 
                                      np.sqrt(rfr_scores), np.sqrt(ada_rfr_scores)], 
                  'Testing Scores' : [np.sqrt(lin_r_test_scores), np.sqrt(dtr_test_scores), 
                                      np.sqrt(ada_dtr_test_scores), np.sqrt(rfr_test_scores), 
                                      np.sqrt(ada_rfr_test_scores)]
                 })
    
d['Variance'] = d['Testing Scores'] - d['Training Scores']

d


,Method,Training Scores,Testing Scores,Variance
0,Linear Regression,3.237209,3.911993,0.674783
1,Decision Trees,0.000000,5.576838,5.576838
2,Adaboost (Decision Trees),0.160764,4.488437,4.327673
3,Random Forest,1.603730,4.346198,2.742469
4,Adaboost (Random Forest),1.015702,4.182508,3.166806


In my opinion, it seems like Linear Regression or Random Forest is the way to go. Not only did they score the highest in the test set but the variance between the models training and testing set was minimal. If using a randomized search, it may be possible to find parameters that best fit the random forest model and be able to better score than the linear regression.


For my next part of this project I will be looking more closely on the player level, and will either use TensorFlow or Keras for the machine learning portion.


In [ ]:
import pickle
with open('hockey_model.pkl', 'wb') as file:
    pickle.dump(lin_r, file)